In [ ]:
%cd /home/adityasidharta/git/shopee_data_science
%env PROJECT_PATH /home/adityasidharta/git/shopee_data_science

In [ ]:
import os
import pandas as pd

from utils.envs import *
from utils.common import create_directory
from pathlib import Path
from tqdm import tqdm

import fastai
from fastai import *
from fastai.vision import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os
from sklearn.model_selection import train_test_split
from utils.envs import *
import string
from utils.common import create_directory

from pathlib import Path
from category_encoders import OrdinalEncoder

In [ ]:
root_folder = '/home/adityasidharta/git/shopee_data'

In [ ]:
beauty_dev_folder = os.path.join(root_folder, 'beauty_dev')
beauty_val_folder = os.path.join(root_folder, 'beauty_val')
beauty_test_folder = os.path.join(root_folder, 'beauty_test')

fashion_dev_folder = os.path.join(root_folder, 'fashion_dev')
fashion_val_folder = os.path.join(root_folder, 'fashion_val')
fashion_test_folder = os.path.join(root_folder, 'fashion_test')

mobile_dev_folder = os.path.join(root_folder, 'mobile_dev')
mobile_val_folder = os.path.join(root_folder, 'mobile_val')
mobile_test_folder = os.path.join(root_folder, 'mobile_test')

In [ ]:
beauty_dev = pd.read_csv(beauty_dev_repo)
beauty_val = pd.read_csv(beauty_val_repo)
beauty_train = pd.read_csv(beauty_train_repo)
beauty_test = pd.read_csv(beauty_test_repo)

fashion_dev = pd.read_csv(fashion_dev_repo)
fashion_val = pd.read_csv(fashion_val_repo)
fashion_train = pd.read_csv(fashion_train_repo)
fashion_test = pd.read_csv(fashion_test_repo)

mobile_dev = pd.read_csv(mobile_dev_repo)
mobile_val = pd.read_csv(mobile_val_repo)
mobile_train = pd.read_csv(mobile_train_repo)
mobile_test = pd.read_csv(mobile_test_repo)

In [ ]:
path = Path('/home/adityasidharta/git/shopee_data')

In [ ]:
def fix_image_path(input_df, relative_path):
    df = input_df.copy()
    for idx in tqdm(range(len(input_df))):
        filename = df.at[idx, 'image_path'].split('/')[1]
        if not filename.endswith('.jpg'):
            filename = filename + '.jpg'
        final_filename = os.path.join(relative_path, filename)
        df.at[idx, 'image_path'] = final_filename
    return df

In [ ]:
beauty_dev = fix_image_path(beauty_dev, 'beauty_dev')
beauty_val = fix_image_path(beauty_val, 'beauty_val')
beauty_test = fix_image_path(beauty_test, 'beauty_test')

fashion_dev = fix_image_path(fashion_dev, 'fashion_dev')
fashion_val = fix_image_path(fashion_val, 'fashion_val')
fashion_test = fix_image_path(fashion_test, 'fashion_test')

mobile_dev = fix_image_path(mobile_dev, 'mobile_dev')
mobile_val = fix_image_path(mobile_val, 'mobile_val')
mobile_test = fix_image_path(mobile_test, 'mobile_test')

In [ ]:
column = 'Brand'
train_df = mobile_train
test_df = mobile_test
test_folder = 'mobile_test'

In [ ]:
train_df = mobile_dev[['image_path', column]]
train_df.columns = ['name', 'label']
train_df = train_df[~pd.isnull(train_df.label)]
train_df['label'] = train_df['label'].astype(str)

In [ ]:
test_df = mobile_test[['image_path']].astype(str)
test_df.columns = ['name']

In [ ]:
tfms = get_transforms()

In [ ]:
data = ImageDataBunch.from_df(path, df=train_df, size = 224)
data.normalize()

In [ ]:
learn = create_cnn(data, models.alexnet, metrics=accuracy)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit(1)

In [ ]:
learn.export()

In [ ]:
test = ImageList.from_folder(path/'mobile_test')
final_learn = load_learner(path, test=test)

In [ ]:
train_items = learn.data.label_list.train.items
val_items = learn.data.label_list.valid.items
test_items = final_learn.data.label_list.test.items

In [ ]:
print(len(train_items))
print(len(val_items))
print(len(test_items))

In [ ]:
train_preds = learn.get_preds(ds_type=DatasetType.Fix)
val_preds = learn.get_preds(ds_type=DatasetType.Valid)
test_preds = final_learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
class_dict = true_dict = {key: value for key, value in enumerate(learn.data.classes)}

In [ ]:
items = train_items.copy()

In [ ]:
label_train_df = pd.DataFrame({
    'name': train_items
})

label_train_df['name'] = label_train_df['name'].apply(lambda x : 'mobile_dev/' + x.split('/')[-1])
label_train_df = label_train_df.merge(train_df, on='name', how='left', validate='1:1')

In [ ]:
label_val_df = pd.DataFrame({
    'name': val_items
})

label_val_df['name'] = label_val_df['name'].apply(lambda x : 'mobile_dev/' + x.split('/')[-1])
label_val_df = label_val_df.merge(train_df, on='name', how='left', validate='1:1')

In [ ]:
label_test_df = pd.DataFrame({
    'name': test_items
})

In [ ]:
preds = train_preds
label = label_train_df['label']


from sklearn.metrics import accuracy_score

argmax_preds = to_np(preds[0]).argmax(axis=1)
final_preds = np.vectorize(class_dict.get)(argmax_preds)
accuracy_score(label, final_preds)

In [ ]:
preds = val_preds
label = label_val_df['label']


from sklearn.metrics import accuracy_score

argmax_preds = to_np(preds[0]).argmax(axis=1)
final_preds = np.vectorize(class_dict.get)(argmax_preds)
accuracy_score(label, final_preds)

In [ ]:
pred_df = pd.DataFrame(to_np(test_preds[0]), columns = learn.data.classes)

In [ ]:
pred_df